In [1]:
import pyspark
print(pyspark.__version__)

import findspark
print(findspark.__version__)

3.2.0
1.4.2


In [2]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('Titanic Data').getOrCreate()

In [9]:
spark()

#instead use spark
# as per SO_Q :https://stackoverflow.com/questions/65375537/function-object-has-no-attribute-read-on-jupyter 

TypeError: 'SparkSession' object is not callable

In [10]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("train.csv"))

In [11]:
df

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [13]:
df.head(5)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S'),
 Row(PassengerId='4', Survived='1', Pclass='1', Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35', SibSp='1', Parch='0', Ticket='113803', Fare='53.1', Cabin='C123', Embarked='S'),
 Row(PassengerId='5', Survived='0', Pclass='3', Name='Allen, Mr. William Henry', Sex='male', Age='35', SibSp='0', Parch='0', Ticket='373450', Fare='8.05', Cabin=None, Embarked='S')]

In [14]:
df.toPandas()
# DO NOT attempt this with LARGE data because
# all data will be loaded into the memory
# and cause memory issues!!

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13,None,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S
888,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,None,1,2,W./C. 6607,23.45,None,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C


In [20]:
# info about data

print('Number of rows:',df.count())

Number of rows: 891


In [25]:
print('Name of columns:',df.columns)

Name of columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [22]:
print('Types of columns:', df.dtypes)

Types of columns: [('PassengerId', 'string'), ('Survived', 'string'), ('Pclass', 'string'), ('Name', 'string'), ('Sex', 'string'), ('Age', 'string'), ('SibSp', 'string'), ('Parch', 'string'), ('Ticket', 'string'), ('Fare', 'string'), ('Cabin', 'string'), ('Embarked', 'string')]


In [28]:
df.describe().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
4,max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,93.5,T,S


### Data Preparation

In [29]:
#some dtypes are string but should be int or float
# like survived,Pclass,Age , Fare


# so by casting them Im changing their dtypes

from pyspark.sql.functions import col

dataset = df.select(col('Survived').cast('float'),
                   col('PClass').cast('float'),
                   col('Sex'),
                   col('Age').cast('float'),
                   col('Fare').cast('float'),
                   col('Embarked'))

dataset.show()

+--------+------+------+----+-------+--------+
|Survived|PClass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   3.0|  male|null| 8.4583|       Q|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|   16.7|       S|
|     1.0|   1.0|female|58.0|  26.55|       S|
|     0.0|   3.0|  male|20.0|   8.05|       S|
|     0.0|   3.0|  male|39.0| 31.275|       S|
|     0.0|   3.0|female|14.0| 7.8542|       S|
|     1.0|   2.0|female|55.0|   16.0|       S|
|     0.0|   3.0|  male| 2.0| 29.125|       Q|
|     1.0|   2.0|  male|null|   13.0|       S|
|     0.0|   

In [30]:
from pyspark.sql.functions import isnull,when,count,col

dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|PClass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



#30 shows how to find null values in dataset
https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe

In [31]:
# remove null values

dataset = dataset.replace('?',None).dropna(how='any')

# dataset = dataset.replace('?',None).dropna(how='all')


In [32]:
# printing dtypes again
df.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [34]:
# using StringIndexer of Spark encode Sex and Embarked columns as they are non-numerics

#https://spark.apache.org/docs/latest/ml-features#stringindexer

In [39]:
from pyspark.ml.feature import StringIndexer

dataset = StringIndexer(
            inputCol='Sex',
            outputCol='Gender',
            handleInvalid='keep').fit(dataset).transform(dataset)

In [40]:
dataset = StringIndexer(
    inputCol='Embarked',
    outputCol='Boarded',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [41]:
dataset.show()

+--------+------+------+----+-------+--------+------+-------+
|Survived|PClass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
|     1.0|   3.0|female|26.0|  7.925|       S|   1.0|    0.0|
|     1.0|   1.0|female|35.0|   53.1|       S|   1.0|    0.0|
|     0.0|   3.0|  male|35.0|   8.05|       S|   0.0|    0.0|
|     0.0|   1.0|  male|54.0|51.8625|       S|   0.0|    0.0|
|     0.0|   3.0|  male| 2.0| 21.075|       S|   0.0|    0.0|
|     1.0|   3.0|female|27.0|11.1333|       S|   1.0|    0.0|
|     1.0|   2.0|female|14.0|30.0708|       C|   1.0|    1.0|
|     1.0|   3.0|female| 4.0|   16.7|       S|   1.0|    0.0|
|     1.0|   1.0|female|58.0|  26.55|       S|   1.0|    0.0|
|     0.0|   3.0|  male|20.0|   8.05|       S|   0.0|    0.0|
|     0.0|   3.0|  male|39.0| 31.275|       S|   0.0|    0.0|
|     0.

Above StringIndexer was applied to Embarked and Sex columns.

This created 2 new columns as Gender and Boarded that contains same info as Sex and Embarked but difference is that they contain numerics.

In [43]:
# check dtypes

dataset.dtypes

# notice the sex->Gender and embarked->Boarder columns

[('Survived', 'float'),
 ('PClass', 'float'),
 ('Sex', 'string'),
 ('Age', 'float'),
 ('Fare', 'float'),
 ('Embarked', 'string'),
 ('Gender', 'double'),
 ('Boarded', 'double')]

In [44]:
# so now remove string cols

dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

dataset.show()

+--------+------+----+-------+------+-------+
|Survived|PClass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|
|     0.0|   1.0|54.0|51.8625|   0.0|    0.0|
|     0.0|   3.0| 2.0| 21.075|   0.0|    0.0|
|     1.0|   3.0|27.0|11.1333|   1.0|    0.0|
|     1.0|   2.0|14.0|30.0708|   1.0|    1.0|
|     1.0|   3.0| 4.0|   16.7|   1.0|    0.0|
|     1.0|   1.0|58.0|  26.55|   1.0|    0.0|
|     0.0|   3.0|20.0|   8.05|   0.0|    0.0|
|     0.0|   3.0|39.0| 31.275|   0.0|    0.0|
|     0.0|   3.0|14.0| 7.8542|   1.0|    0.0|
|     1.0|   2.0|55.0|   16.0|   1.0|    0.0|
|     0.0|   3.0| 2.0| 29.125|   0.0|    2.0|
|     0.0|   3.0|31.0|   18.0|   1.0|    0.0|
|     0.0|   2.0|35.0|   26.0|   0.0|    0.0|
|     1.0|   2.0|34.0|   13.0|   0

##### Unlike pandas we need to give spark all in one go.Greedy spark!!
Spark actually works to predict with a column with all the features smashed together into a list-like structure.And you want to predict “Survived”, you need to combine the information of the columns “Pclass”, “Age”, “Fare”, “Gender” and “Boarded” into one column. We normally call that column features.

This new column features contain the same information from all of our features but in a list-like object

In [45]:
# hence we use vectorassembler

For vector assembler:
https://medium.com/@nutanbhogendrasharma/feature-transformer-vectorassembler-in-pyspark-ml-feature-part-3-b3c2c3c93ee9

In [46]:
# combining all features as into single feature column using 
# vectorAssembler

required_features = ['PClass',
                    'Age',
                    'Fare',
                    'Gender',
                    'Boarded']

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=required_features,
                           outputCol='features')

transformed_data = assembler.transform(dataset)

In [48]:
# now the dataset looks like

dataset.show(5)

+--------+------+----+-------+------+-------+
|Survived|PClass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|
+--------+------+----+-------+------+-------+
only showing top 5 rows



## ML Part now

1. Split data
2. Build model
3. Fit model
4. evaluate model

In [49]:
#splitting
(training_data, test_data)=transformed_data.randomSplit([0.75,0.25])

In [51]:
#building model using RandomForest classifier

from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    labelCol='Survived',
    featuresCol='features',
    maxDepth=5)

In [52]:
# fit model

model = rf.fit(training_data)

# this gives us transformer

In [53]:
# predict

predictions = model.transform(test_data)

In [61]:
#evaluate the model
# using accuracy

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived',
    predictionCol='prediction',
    metricName='accuracy')

In [62]:
# print out accuracy

accuracy = evaluator.evaluate(predictions)
print('Test accuracy is:', accuracy)

Test accuracy is: 0.8870056497175142
